In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette('muted')
sns.set_color_codes('muted')
sns.set_style('white')
%config InlineBackend.figure_format = 'retina'
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import OneHotEncoder

Dataset is downloaded from >100K used cars postings from the British used cars
https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes

In [4]:
df = pd.read_csv('/content/cars.csv')
df
df['tax'].fillna(df['tax(£)'],inplace=True)
del df['tax(£)']
m = df[df.tax.notna()].tax.mean()
df['tax'].fillna(m,inplace=True)
mm = df[df.mpg.notna()].tax.mean()
df['mpg'].fillna(mm,inplace=True)

df.dropna(inplace= True)
df.isna().sum()

brand           0
model           0
year            0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
price           0
dtype: int64

In [5]:
df.drop_duplicates()
df.drop(df[(df['year']<1996) | (df['year']>2021)].index,inplace = True)

**Data preparation and normalization**

In [6]:
df.describe()

,year,mileage,tax,mpg,engineSize,price
count,108476.000000,108476.000000,108476.000000,108476.000000,108476.000000,108476.000000
mean,2017.098962,23022.670065,120.288342,60.779815,1.661567,16889.836692
std,2.114219,21168.886584,60.355077,23.916652,0.556764,9754.566972
min,1996.000000,1.000000,0.000000,0.300000,0.000000,450.000000
25%,2016.000000,7490.000000,120.293518,47.100000,1.200000,10230.000000
50%,2017.000000,17263.000000,145.000000,56.500000,1.600000,14698.000000
75%,2019.000000,32235.000000,145.000000,65.700000,2.000000,20932.250000
max,2020.000000,323000.000000,580.000000,470.800000,6.600000,159999.000000


In [7]:
'''def data_norm(df,*cols):
    df_n = df.copy()
    for col in cols:
        ma = df_n[col].max()
        mi = df_n[col].min()
        df_n[col + '_n'] = (df_n[col] - mi) / (ma - mi)
    return(df_n)
df_n = data_norm(df,'mileage','tax','mpg','engineSize')
df_n
'''
def data_normz(df,*cols):
    df_n = df.copy()
    for col in cols:
        u = df_n[col].mean()
        std = df_n[col].std()
        df_n[col + '_zn'] = (df_n[col] - u) / std
    return(df_n)
df_n = data_normz(df,'year','mileage','tax','mpg','engineSize')
df_n


,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize,price,year_zn,mileage_zn,tax_zn,mpg_zn,engineSize_zn
0,ford,Fiesta,2017.0,Manual,16105,Petrol,125.0,54.3,1.2,7770,-0.046808,-0.326785,0.078066,-0.270933,-0.829017
1,vw,Golf,2019.0,Manual,9000,Petrol,150.0,47.1,1.5,17250,0.899168,-0.662419,0.492281,-0.571979,-0.290189
2,audi,A3,2015.0,Manual,49441,Diesel,0.0,83.1,1.6,10299,-0.992784,1.247979,-1.993011,0.933249,-0.110579
3,skoda,Superb,2018.0,Semi-Auto,6241,Petrol,145.0,36.7,1.5,18798,0.426180,-0.792752,0.409438,-1.006822,-0.290189
4,ford,Galaxy,2017.0,Manual,27203,Diesel,125.0,56.5,2.0,18170,-0.046808,0.197475,0.078066,-0.178947,0.607858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108485,toyota,Yaris,2017.0,Manual,10000,Petrol,145.0,56.5,1.5,11650,-0.046808,-0.615180,0.409438,-0.178947,-0.290189
108486,vauxhall,Viva,2016.0,Manual,29601,Petrol,20.0,62.8,1.0,5991,-0.519796,0.310755,-1.661639,0.084468,-1.188236
108487,vauxhall,Corsa,2019.0,Manual,8203,Petrol,150.0,43.5,1.4,10498,0.899168,-0.700068,0.492281,-0.722501,-0.469798
108488,vauxhall,Crossland X,2018.0,Manual,12512,Petrol,145.0,54.3,1.2,11798,0.426180,-0.496515,0.409438,-0.270933,-0.829017


In [8]:
df_n.describe()

,year,mileage,tax,mpg,engineSize,price,year_zn,mileage_zn,tax_zn,mpg_zn,engineSize_zn
count,108476.000000,108476.000000,108476.000000,108476.000000,108476.000000,108476.000000,1.084760e+05,1.084760e+05,1.084760e+05,1.084760e+05,1.084760e+05
mean,2017.098962,23022.670065,120.288342,60.779815,1.661567,16889.836692,-1.860220e-14,1.096737e-16,-3.989495e-13,1.020098e-13,-3.923841e-13
std,2.114219,21168.886584,60.355077,23.916652,0.556764,9754.566972,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,1996.000000,1.000000,0.000000,0.300000,0.000000,450.000000,-9.979555e+00,-1.087524e+00,-1.993011e+00,-2.528774e+00,-2.984330e+00
25%,2016.000000,7490.000000,120.293518,47.100000,1.200000,10230.000000,-5.197958e-01,-7.337500e-01,8.576007e-05,-5.719787e-01,-8.290170e-01
50%,2017.000000,17263.000000,145.000000,56.500000,1.600000,14698.000000,-4.680783e-02,-2.720819e-01,4.094379e-01,-1.789471e-01,-1.105794e-01
75%,2019.000000,32235.000000,145.000000,65.700000,2.000000,20932.250000,8.991681e-01,4.351825e-01,4.094379e-01,2.057221e-01,6.078582e-01
max,2020.000000,323000.000000,580.000000,470.800000,6.600000,159999.000000,1.372156e+00,1.417067e+01,7.616785e+00,1.714371e+01,8.869890e+00


In [ ]:
df['tax'].fillna(df['tax(£)'],inplace=True)
del df['tax(£)']
m = df[df.tax.notna()].tax.mean()
df['tax'].fillna(m,inplace=True)
mm = df[df.mpg.notna()].tax.mean()
df['mpg'].fillna(mm,inplace=True)

df.dropna(inplace= True)
df.isna().sum()

In [49]:
dft = pd.read_csv('/content/cars_test.csv')

dft['tax'].fillna(dft['tax(£)'],inplace=True)

mt = dft[dft.tax.notna()].tax.mean()
dft['tax'].fillna(mt,inplace=True)

mmt = dft[dft.mpg.notna()].mpg.mean()
dft['mpg'].fillna(mmt,inplace=True)

del dft['tax(£)']



dft.isna().sum()
dft.describe()

,year,mileage,tax,mpg,engineSize
count,9605.000000,9605.000000,9605.000000,9605.000000,9605.000000
mean,2017.107860,23027.434774,120.627565,60.789241,1.666278
std,2.101453,21126.634436,59.651407,24.289582,0.562450
min,1996.000000,1.000000,0.000000,19.600000,0.000000
25%,2016.000000,7575.000000,120.627565,47.100000,1.200000
50%,2017.000000,17052.000000,145.000000,55.400000,1.600000
75%,2019.000000,32125.000000,145.000000,65.700000,2.000000
max,2020.000000,178000.000000,580.000000,470.800000,6.200000


In [13]:
df_nt = data_normz(dft,'year','mileage','tax','mpg','engineSize')
df_nt.describe()

,year,mileage,tax,mpg,engineSize,year_zn,mileage_zn,tax_zn,mpg_zn,engineSize_zn
count,9605.000000,9605.000000,9605.000000,9605.000000,9605.000000,9.605000e+03,9.605000e+03,9.605000e+03,9.605000e+03,9.605000e+03
mean,2017.107860,23027.434774,120.627565,55.106920,1.666278,6.964404e-15,-6.287989e-18,-6.196143e-15,5.866720e-15,-3.889475e-14
std,2.101453,21126.634436,59.651407,15.809105,0.562450,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,1996.000000,1.000000,0.000000,19.600000,0.000000,-1.004441e+01,-1.089924e+00,-2.022208e+00,-2.245979e+00,-2.962534e+00
25%,2016.000000,7575.000000,120.627565,47.100000,1.200000,-5.271879e-01,-7.314196e-01,-5.479329e-15,-5.064752e-01,-8.290120e-01
50%,2017.000000,17052.000000,145.000000,55.106920,1.600000,-5.132663e-02,-2.828389e-01,4.085811e-01,2.112423e-14,-1.178380e-01
75%,2019.000000,32125.000000,145.000000,61.400000,2.000000,9.003960e-01,4.306207e-01,4.085811e-01,3.980668e-01,5.933361e-01
max,2020.000000,178000.000000,580.000000,470.800000,6.200000,1.376257e+00,7.335412e+00,7.700949e+00,2.629454e+01,8.060664e+00


In [11]:
numeric_features = ['year', 'mileage_zn','mpg_zn','tax_zn', 'engineSize']
mapper = DataFrameMapper(
    [([feature], SimpleImputer()) for feature in numeric_features] + [
    (['model'], OneHotEncoder(sparse=False, handle_unknown='ignore')),
        (['transmission'], OneHotEncoder(sparse=False, handle_unknown='ignore')),
        (['fuelType'], OneHotEncoder(sparse=False,drop='first')),
       

], df_out=True)

pipe = make_pipeline(
    mapper,
    LinearRegression()
)
pipe.fit(df_n, df.price)
print()
print('MSE',mean_squared_error(df.price, pipe.predict(df_n), squared=False))


MSE 3685.70930702116


In [14]:
p = pipe.predict(df_nt)
p[p<0] = 3750
dft['price'] = p

**CatBoost implementation**

In [15]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 23 kB/s 


In [17]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

In [25]:
y = df['price']
del df['price']

In [26]:
x_train, x_test, y_train, y_test = train_test_split(df, y, 
                                                    train_size=0.8,
                                                    random_state=42)

In [28]:
feature_names = list(x_train.columns)
cat_features = [column for column, dtype in x_train.dtypes.items() if dtype==object]

x_train[cat_features] = x_train[cat_features].fillna('NA')
x_test[cat_features] = x_test[cat_features].fillna('NA')

train_pool = Pool(x_train, y_train, feature_names=feature_names, cat_features=cat_features)
test_pool = Pool(x_test, y_test, feature_names=feature_names, cat_features=cat_features)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [34]:
model = CatBoostRegressor(loss_function='RMSE')

In [ ]:
model.fit(train_pool, eval_set=test_pool,  early_stopping_rounds = 20)

In [36]:
from sklearn.metrics import mean_squared_error

In [60]:
dft = pd.read_csv('/content/cars_test.csv')

dft['tax'].fillna(dft['tax(£)'],inplace=True)
del dft['tax(£)']
m = dft[dft.tax.notna()].tax.mean()
dft['tax'].fillna(m,inplace=True)
mm = dft[dft.mpg.notna()].tax.mean()
dft['mpg'].fillna(mm,inplace=True)
test_id = dft['id']
del dft['id']
dft.dropna(inplace= True)
dft.isna().sum()
dft

,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,vw,Polo,2019.0,Manual,2999,Petrol,145.0,52.3,1.0
1,audi,Q5,2019.0,Automatic,6929,Petrol,145.0,34.0,3.0
2,vauxhall,Corsa,2015.0,Manual,37500,Petrol,30.0,55.4,1.4
3,Mercedes-Benz,S Class,1998.0,Automatic,43534,Petrol,265.0,23.3,6.0
4,Hyundai,Kona,2019.0,Automatic,9500,Hybrid,135.0,52.3,1.6
...,...,...,...,...,...,...,...,...,...
9600,ford,Mondeo,2016.0,Semi-Auto,59601,Diesel,125.0,57.7,2.0
9601,audi,A3,2015.0,Manual,89060,Diesel,30.0,67.3,2.0
9602,vw,Polo,2019.0,Manual,1267,Petrol,150.0,49.6,1.0
9603,vw,Golf,2019.0,Semi-Auto,1658,Petrol,145.0,36.2,2.0


In [61]:
feature_names = list(dft.columns)
cat_features = [column for column, dtype in dft.dtypes.items() if dtype==object]

dft[cat_features] = dft[cat_features].fillna('NA')
test_pool = Pool(dft, feature_names=feature_names, cat_features=cat_features)

In [62]:
pred = model.predict(dft)
pred

array([14860.24200832, 46107.8472211 ,  6329.21719704, ...,
       14981.42532837, 30514.22705427, 13268.10266883])

In [66]:
newDf = pd.DataFrame()
newDf['id'] = test_id
newDf['price'] = pred
newDf

,id,price
0,NMBZ22VY,14860.242008
1,E6LR9TEY,46107.847221
2,LSKER65V,6329.217197
3,99QY0FR4,21056.011628
4,7W1OP1NO,20810.686406
...,...,...
9600,LI848AQ1,11902.695003
9601,ANKJJJKS,9831.866494
9602,5UJ8YW1R,14981.425328
9603,9O04KPDP,30514.227054


In [67]:
newDf.to_csv('./submiss.csv', index=False)